In [ ]:
'''Project description
Creating a program to pull up country data from variable sources

Submission 1:
1. managed to query WB data for a user specified country
2. Create a user interface

Submission 2:
1. Pull in more informaion for other sources -- Google trend API failed
2. Include another country in the query as benchmark (for user to make sense of the data)
3. Improve the user interface
'''

In [1]:
##### Part A

#Prep-->package installation

!pip install wbpy
import wbpy
from pprint import pprint

!pip install pycountry
import pycountry

In [ ]:
##### Part B

# on Mac (in jupyter notebook): if installation fails, you need to: 
# 1. adjust the installation directory usign the following code
# 2. restart the kernel
# 3. reinstall packages
import sys, os
sys.path.append('/Volumes/Seagate/Applications/anaconda3/lib/python3.8/site-packages')
os.getcwd()
sys.executable
conda install ipykernel --name Python3, python -m ipykernel install


In [2]:
##### Part C

#### Prep-->define indicator list

#https://github.com/mattduck/wbpy
api = wbpy.IndicatorAPI()

# Indicator list

#######  low freq
#GDP (real, Nominal; level and growth rates)
ngdp='ny.gdp.mktp.cd' #usd
cgdp='NY.GDP.MKTP.KD' #2010 usd
#Population
total_population = "SP.POP.TOTL"
#Current account balance (USD)
cab='BN.CAB.XOKA.CD'
#Unemployment - %of total labor force
unemployment='SL.UEM.TOTL.NE.ZS'
#Doing business index (1=most friendly)
dbi='IC.BUS.EASE.XQ'


# Combined indicator list
i_list=[ngdp,cgdp,total_population,cab,unemployment,dbi]
n_list=['Nominal GDP in USD','Real GDP in 2010 USD','Total population','Current account balance in USD','Unemployment rate','Doing business index (1=most friendly)']

In [ ]:
##### Part D


from tkinter import *

        
top_widget=Tk()
top_widget.geometry('550x400+700+300')
top_widget.title('Find country info')

# instruction label 1
label1=Label(top_widget,text='Enter a country name',bg='gold',fg='blue').place(x=25,y=25)
# instruction label 2
label2=Label(top_widget,text='Enter a reference country',bg='gold',fg='blue').place(x=25,y=55)

# User input 1 ---- = country
box1=Entry(top_widget)
box1.focus()
box1.place(x=200,y=25)
# User input 2 ---- = home_country
box2=Entry(top_widget)
box2.focus()
box2.place(x=200,y=55)

# Output textbox
label3=Label(top_widget,text='Output',bg='white',fg='blue')
label3.place(x=25,y=145)
label3.config(width=55)
# Warnings textbox
label4=Label(top_widget,text='Warnings',bg='white',fg='blue')
label4.place(x=25,y=360)

#Fuzzy match result textbox
label5=Label(top_widget,text='Did you mean',bg='gold',fg='blue').place(x=25,y=85)
label6=Label(top_widget,text='Identified inputs',bg='white',fg='blue')
label6.place(x=200,y=85)
# check inputs
def checkinput(a,b):
    c_matched=pycountry.countries.search_fuzzy(a)
    c_matched1=pycountry.countries.search_fuzzy(b)
    match=c_matched[0] 
    match1=c_matched1[0] 
    name1,name2=match.name,match1.name
    label6['text'] = (name1,name2)
def call_checkinput():
    country=box1.get()
    home_country=box2.get()
    checkinput(country,home_country)
btn1=Button(top_widget,text='Check inputs',command=call_checkinput).place(x=400,y=40)

def get_cntydata(country,home_country):
    c_matched=pycountry.countries.search_fuzzy(country)  #fuzzy match user input of a country to country codes
    c_matched1=pycountry.countries.search_fuzzy(home_country)
    match=c_matched[0]      
    c_code=match.alpha_2

    match1=c_matched1[0]      
    c_code1=match1.alpha_2

    flag=''

    resultA={}
    resultB={}
    temp={}
    for item in i_list:
        item=api.get_dataset(item, [c_code,c_code1], date="2018:2020")
        temp=item.as_dict()
        resultA[item]=temp[c_code]
        resultB[item]=temp[c_code1] #complete the API query by country

    #         restructure the returned results
    #country of interest
    result1={}
    result2={}
    flag={}
    for code,item,name in zip(i_list,resultA.values(),n_list): 
        result2['2020']=item['2020']
        result2['2019']=item['2019']
        result2['2018']=item['2018']
    #  query the last three years and keep the most recent number; if no values returned, print "no updates since 2017"
        if result2['2020'] is None:
            if result2['2019'] is None:
                if result2['2018'] is None:
                    result1[name]=0
                    flag[name]="Warning: Variable "+name+ " has no updates since 2017"
                else:
                    result1[name]=result2['2018']
            else:
                result1[name]=result2['2019']
        else:
            result1[name]=result2['2020']
    # adjust units 
    result6={}
    for name in n_list:
        if result1[name]>10**12:
            u=round(result1[name]/10**12,2)
            result6[name]=str(u)+"tri."
        elif result1[name]>10**9:
            u=round(result1[name]/10**9,2)
            result6[name]=str(u)+"bil."
        elif result1[name]>10**6:
            u=round(result1[name]/10**6,2)
            result6[name]=str(u)+"mil."
        elif result1[name]>10**3:
            u=round(result1[name]/10**3,2)
            result6[name]=str(u)+"th."
        else:
            result6[name]=result1[name]
    #for unemployment rate
    result6['Unemployment rate']=str(round(result1['Unemployment rate'],2)) + " %"
    
    #country for reference
    result3={}
    result4={}
    flag2={}
    for code,item,name in zip(i_list,resultB.values(),n_list):
        result4['2020']=item['2020']
        result4['2019']=item['2019']
        result4['2018']=item['2018']
        if result4['2020'] is None:
            if result4['2019'] is None:
                if result4['2018'] is None:
                    result3[name]=0
                    flag2[name]="Warning: Variable "+name+ " has no updates since 2017"
                else:
                    result3[name]=result4['2018']
            else:
                result3[name]=result4['2019']
        else:
            result3[name]=result4['2020']    
            
#  Compare the data between the two countries
    result5={}
    multi=n_list[:4]  # for ngdp,cgdp,total_population, compare by dividing the two countries' values
    for name in multi:
        try:
            t= result1[name]/result3[name]
            result5[name]=str(round(t,2)) + ' times of ' + match1.name + "'s"
        except ValueError:
            result5[name]=''

    l_s=n_list[4:]  # for unemployment,doing business index, compare the two countries' values
    for name in l_s:
        if result1[name]>=result3[name]:
            result5[name]='larger than ' + match1.name + "'s"
        else:
            result5[name]='smaller than ' + match1.name + "'s"
    #return result1,flag,result5, flag2
    #print(result1,flag.values(),result5, flag2.values())
    label3['text'] = '\n'.join('{} : {} ; {}'.format(k, d,y) for k, d,y in zip(result6.keys(),result6.values(),result5.values()))
    label4.config(text=flag.values(),wraplength=400)
    
def call_get_cntydata():
    country=box1.get()
    home_country=box2.get()
    get_cntydata(country,home_country)
    #label3['text'] = '\n'.join('{} : {}'.format(k, d) for k, d in result1.items())
    #label4.config(text=flag)

btn=Button(top_widget,text='Get country information',command=call_get_cntydata).place(x=170,y=110)


top_widget.mainloop()

http://api.worldbank.org/v2/countries/CHN;USA/indicators/ny.gdp.mktp.cd?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN;USA/indicators/NY.GDP.MKTP.KD?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN;USA/indicators/SP.POP.TOTL?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN;USA/indicators/BN.CAB.XOKA.CD?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN;USA/indicators/SL.UEM.TOTL.NE.ZS?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN;USA/indicators/IC.BUS.EASE.XQ?date=2018%3A2020&format=json&per_page=10000


In [13]:
##### Part E
# Failed to generate meaningful and appropriate conversation topics with Google trend API
# 
# Install
!pip install pytrends
# test example - china
country='china'
#https://towardsdatascience.com/google-trends-api-for-python-a84bc25db88f
import pandas as pd
from pytrends.request import TrendReq
pytrend = TrendReq()
#get google keywordsuggestions
keywords = pytrend.suggestions(keyword=country)
df = pd.DataFrame(keywords)
df.drop(columns='mid')

pytrend.build_payload(kw_list=[country])
related_queries = pytrend.related_queries()
related_queries[country]['rising']


,query,value
0,coronavirus china,267200
1,covid china,93000
2,coronavirus in china,65850
3,corona virus,49900
4,corona virus china,49200
5,covid 19 china,43150
6,corona in china,21450
7,video bokeh china mp3,20250
8,corona china,12250
9,video bokeh china,11750
